## PHYS 305 - Solutions to Problem Set 3
### Fall 2020, Prof. Elisabeth Krause

## Problem 1 - Romberg Integration
 Write a Python function `romberg(func, a, b, eps, *P)` which computes the integral of the
  given function between the given limits, using Romberg integration to obtain a solution to within
  an error of approximately \texttt{eps}. Use this function to solve the following problem.

  The flux leaving a square centimeter of a blackbody radiator at temperature $T$ is
  $F = \sigma T^4$ (where $\sigma$ is the Stephan-Boltzmann constant). This is the result of
  integrating the flux from a Plank spectrum over all frequencies
  \begin{equation*}
    \sigma T^4 = \pi \int_0^\infty B_\nu(T)\ d\nu
  \end{equation*}
  where the Planck function is
  \begin{equation*}
    B_\nu(T) = \frac{2h\nu^3}{c^2}\left\{e^{h\nu/kt}-1\right\}^{-1}
  \end{equation*}
  Start by reducing the integral to
  \begin{equation*}
    \int_0^\infty \frac{x^3\ dx}{e^x - 1}
  \end{equation*}
  by a change of variable. Use your Romberg integration function to obtain the value of this
  integral and evaluate $\sigma$ in the units of your choice. Check your answer against the correct
  value.

_Note that this and the next problem present integrals with several mathematical difficulties.  The
  integral above is singular at the origin and both this and the next problem are integrals with an
  infinite range._


### Answer
In this problem, not only did you need to implement Romberg integration, you also needed to figure out how to deal with the singularity at the origin and the infinite limits of integration.

To deal with the singularity, we can use l'Hopital's rule to show that the limit of the integrand at x=0 is zero: setting $f(x) = x^3$ and $g(x) = e^x-1$
$$ \lim_{x\rightarrow 0} \frac{f'(x)}{g'(x)} = \lim_{x\rightarrow 0} \frac{3x^2}{e^x} = 0$$
converges, hence $$\lim_{x\rightarrow 0} \frac{f(x)}{g(x)}=0\,,$$
hence I made the function `planck(x)` return 0 if $x=0$.

More interesting are the limits of integration. Two sources of error affect the result of your
calculation. The first is the truncation error left after Richardson extrapolation. The second is the value of the remaining integral from some finite $x_{max}$ to $\infty$.  Because the integrand decreases relatively rapidly with $x$, we clearly don't need to go to too large an upper limit of integration before the remaining integral becomes negligibly small. To show this, I not only used Romberg integration to evaluate the integral as requested, I also used it to evaluate the error from not completing the integration to $\infty$.  Since this is only one of two errors, as long as this "remainder integral" is rather smaller than the estimate of the error from Richardson extrapolation, we have an acceptable value for $x_{max}$.

Trying to push things to the breaking point, I asked for 13 digits of relative accuracy. I found that $x_{max}=70$ gave the best answer, but this is clearly fortuitous since the remaining integral at this point is in the 24-th decimal place! Asking for 13 digits of accuracy with only 14 digits of precision in each arithmetic step is clearly asking too much. 

In [1]:
"""
Module to implement Romberg integration

"""
import numpy as np

def nextTrapezoidal(func, xmin, xmax, k, *P):
    """
    function to add intervening points to a previous trapezoidal rule result
    """
    n = 2**k
    h = (xmax-xmin)/n
    sum = 0
    for i in range(1, int(n/2)+1):
        sum += func(xmin + (2*i-1)*h, *P)
    return h*sum

def romberg(func, xmin, xmax, epsilon, *P):
    """
    Trapezoidal Rule with Richardson Extrapolation: Romberg's method
    """
    LMAX = 20
    I = np.zeros((LMAX,LMAX))

    # 1-interval trapezoidal approximation
    I[0,0] = 0.5*(xmax-xmin)*(func(xmin, *P) + func(xmax, *P))

    for l in range(1,LMAX):
        # 2**l interval trapezoidal rule approximation
        I[l,0] = 0.5*I[l-1,0] + nextTrapezoidal(func, xmin, xmax, l, *P)

        # do as many Richardson extrapolations as we have data for
        for k in range(1,l+1):
            q = 4**k
            I[l,k] = (q*I[l,k-1] - I[l-1,k-1])/(q-1)

        if abs(I[l,l] - I[l,l-1]) < epsilon * abs(I[l,l-1]): return I[l,l]

    print("rombergIntegrate: failure to converge after", LMAX, "iterations; returning None")
    # returning None will allow one to test for failure
    return None

In [3]:
#Integrate Planck function to obtain Stefan-Boltzmann constant
import numpy as np
from scipy.constants import Stefan_Boltzmann, Boltzmann, speed_of_light, Planck

# Planck function integrand
def planck(x):
    if x==0:           # deal with singularity at origin
        return 0
    else:
        return x**3/(np.exp(x)-1)

xmax = 70     # upper limit to integration
acc = 1e-13   # desired relative accuracy

ans = romberg(planck, 0, xmax, acc)
exact = np.pi**4/15.0

# showing off here -- how to set number of significant figures at runtime
digits = int(-np.log10(acc))+1
print("Integral = {0:.{width}f}".format(ans,width=digits) )
print("   exact = {0:.{width}f}   rel. err.={1:.3e}".format(exact,(ans-exact)/exact,width=digits) )

# estimate the error in the integral above xmax
rest = romberg(planck, xmax, 100, acc)
print("rest of integral~",rest)

print()
ans = ans * 2*np.pi*Boltzmann**4/(Planck**3*speed_of_light**2)
print("SB constant: ", ans)
print("scipy gives: ", Stefan_Boltzmann)

Integral = 6.49393940226688
   exact = 6.49393940226683   rel. err.=7.249e-15
rest of integral~ 1.423711912122581e-25

SB constant:  5.670374419184472e-08
scipy gives:  5.670374419e-08


## Problem 2 - Gaussian Integral
It is reported that Lord Kelvin (of thermodynamic fame) wrote the following equality on the
  board
  \begin{equation*}
    \int_{-\infty}^\infty e^{-x^2}\ dx = \pi^{1/2}
  \end{equation*}
  and then quipped "a mathematician is one to whom _that_ is as obvious as that twice two makes
  four is to you."

  Use the same Romberg integrator you implemented for the previous problem to
  evaluate this integral. Once again, evaluate your answer for accuracy.
  How does the result change with changes in the limits of integration?
  Is there a symmetry in the integral which you can exploit?


### Answer
The symmetry in the integral is in the integrand -- it is a symmetric function of $x$, and
    hence we can take twice the integral from $0$ to $\infty$. Here again, I estimated the remainder
    integral to show that it is much less than the truncation (and probably some roundoff) error. 

In [5]:
def func(x):
    return np.exp(-x**2)

# upper limit of integration
xmax = 6
# desired relative accuracy
acc = 1e-13

ans = 2 * romberg(func, 0, xmax, acc)
exact = np.sqrt(np.pi)
print("integral ={0:.15f}".format(ans) )
relerr = abs((ans-exact)/exact)
print("   exact ={:.15f}      rel err ={:.3e}".format(exact, relerr) )

# this should be well less than the (absolute) error
#   if xmax is large enough
rest = 2 * romberg(func, xmax, 100, acc)
print("rest of integral~", rest)


integral =1.772453850905537
   exact =1.772453850905516      rel err =1.190e-14
rest of integral~ 3.814274020667338e-17


### Problem 3 - Gaussian integration
- Derive the formula for $n=4$ Gaussian quadrature using the
    recipe based on the Legendre polynomials. Up to what degree
    polynomial is the $n=4$ Gaussian quadrature exact for? Use the
    polynomial $p(x)=36 x^7 + x^6 -58 x^5 -3600 x^4 +5 x^3 -x^2 +10^3
    x+1$ to test your expression. What is $\int_{-1}^1p(x)dx$ equal to
    and does it agree with your $n=4$ Gaussian quadrature expression?

- Use the $n=2,3,\,\mathrm{and}\,4$ Gaussian quadrature methods to compute the
    integral $\mathbf{I=\int_{0}^{\pi}x\cos(x)dx}$. The exact value of
    the integral is $\mathbf{I=-2}$. What is the relative error of the
    $\mathbf{n}=4$ Gaussian quadrature? How does it compare to the
    relative error of the $\mathbf{n}=2$ and $\mathbf{n}=3$ Gaussian
    quadrature methods? Does the error converge to 0 as $n$ increases? 

### Answer - Part 1

 The roots of the degreee 4 Legendre polynomial $P_4(x)$ are the solutions of the equation $x^4-6x^2/7 +3/35=0$, which can easily be solved by setting $x^2=y$ and solving for the two roots of the equation $y^2-6y/7 +3/35=0$. The 4 roots of $P_4(x)$ are:
 
 $x_1 = -\sqrt{\frac{3}{7}-\frac{2}{7}\sqrt{\frac{6}{5}}}$, $x_2 = -\sqrt{\frac{3}{7}+\frac{2}{7}\sqrt{\frac{6}{5}}}$, $x_3 = \sqrt{\frac{3}{7}-\frac{2}{7}\sqrt{\frac{6}{5}}}$, $x_4 = \sqrt{\frac{3}{7}+\frac{2}{7}\sqrt{\frac{6}{5}}}$

  The constants $c_i = \int_{-1}^{1} \prod_{j=1,j\neq i}^n \frac{x-x_j}{x_i-x_j} dx$ are
  
 
  $c_1=\frac{1}{36}(18+\sqrt{30})$,   $c_2=\frac{1}{2}(1-\frac{1}{3}\sqrt{\frac{5}{6}})$
  $c_3=\frac{1}{36}(18+\sqrt{30})$,   $c_4=\frac{1}{2}(1-\frac{1}{3}\sqrt{\frac{5}{6}})$

  Thus, the $n=4$ Gaussian quadrature is given by
$$
    \begin{split}
      \int_{-1}^{1} f(x)dx = & \frac{1}{36}(18+\sqrt{30})f\bigg(-\sqrt{\frac{3}{7}-\frac{2}{7}\sqrt{\frac{6}{5}}}\bigg)+\frac{1}{2}(1-\frac{1}{3}\sqrt{\frac{5}{6}})f\bigg(-\sqrt{\frac{3}{7}+\frac{2}{7}\sqrt{\frac{6}{5}}}\bigg) \\
      & + \frac{1}{36}(18+\sqrt{30})f\bigg(\sqrt{\frac{3}{7}-\frac{2}{7}\sqrt{\frac{6}{5}}}\bigg) + \frac{1}{2}(1-\frac{1}{3}\sqrt{\frac{5}{6}})f\bigg(\sqrt{\frac{3}{7}+\frac{2}{7}\sqrt{\frac{6}{5}}}\bigg)
    \end{split}
    $$
  and is exact for any polynomial $f(x)$ of degree 7 or less.

  The integral $\int_{-1}^1 p(x) dx = -\frac{30206}{21}$, which agrees exactly with the $n=4$ Gaussian quadrature.
  
_Note that you need to evaluate the Gaussian quadrature expressions symbolicaly/by hand to obtainthis exact agreement. The numerical evaluation is affected by round-off errors:_


In [13]:
def transform_variable(t, a,b):
    return ((b-a)*t+a+b)/2
def GaussianQ_n4_any_interval(f,a,b,*P):
    n = 4
    #define nodes on [-1,1] interval(derived above)
    t1 = -np.sqrt(3/7-2/7*np.sqrt(6/5))
    t2 = -np.sqrt(3/7+2/7*np.sqrt(6/5))
    t = np.array([t1,t2,-t1,-t2])
    #define coefficients (derived above)
    c1 = 1/36*(18+np.sqrt(30))
    c2 = 1/2*(1-1/3*np.sqrt(5/6))
    c = np.array([c1,c2,c1,c2])
    I = 0
    #now sum up the integral based on the n=4 Gaussian Quadrature formula
    for j in range(n):
        I += (b-a)/2.*c[j]*f(transform_variable(t[j],a,b),*P)
    return I

In [38]:
# define the example function p(x)
# for simplicity, we're not using P to pass parameters this time
def p(x,*P):
    return 36*x**7 + x**6 -58*x**5 -3600*x**4 +5*x**3 -x**2 +1000*x+1

P = ()
exact_result = -30206/21
GQ_4_result = GaussianQ_n4_any_interval(p,-1,1,*P)
print("exact result: %e, n=4 Quadrature: %e, relative error = %e\n"%(exact_result,GQ_4_result,np.abs((exact_result-GQ_4_result))/exact_result))

exact result: -1.438381e+03, n=4 Quadrature: -1.438381e+03, relative error = -4.742284e-16



### Part 2
The integral $I=\int_{0}^{\pi}x\cos(x)dx=\frac{\pi}{2}\int_{-1}^1f\bigg(\frac{\pi}{2}(t+1)\bigg)dt$. Using the $n=4$ Gaussian quadrature
  we find $\bar I=-2.0001242$, while the exact value is $I=-2$. Hence the relative error is $|(\tilde I-I)/I|=6.2\times 10^{-5}$!!!
  Recall the relative error with the $n=3$ Gaussian quadrature was $4.06\times 10^{-3}$. The $n=2$ Gaussian quadrature yields
  $\tilde I=\frac{\pi}{2}\bigg[f\bigg(\frac{\pi}{2}(-\sqrt{3}/3+1)\bigg)+f\bigg(\frac{\pi}{2}(\sqrt{3}/3+1)\bigg)\bigg]=-2.2439504$, i.e., relative error $0.121975$. Thus, the error decreases as $n$ increases.

In [35]:
def func(x,*P):
    return x*np.cos(x) 
GQ_n4 = GaussianQ_n4_any_interval(func,0,np.pi,*P)

In [37]:
#copying code from in-class notebook GaussianQuadrature.ipynb

def GaussianQ_n3_any_interval(f,a,b,*P):
    n = 3
    #define nodes on [-1,1] interval(derived above)
    t = np.array([-np.sqrt(3/5.),0,np.sqrt(3/5.)])
    #define coefficients (derived in GaussianQuadrature.ipynb)
    c = np.array([5./9,8./9,5./9])
    I = 0
    #now sum up the integral based on the n=3 Gaussian Quadrature formula
    for j in range(n):
        I += (b-a)/2.*c[j]*f(transform_variable(t[j],a,b),*P)
    return I
def GaussianQ_n2_any_interval(f,a,b,*P):
    n = 2
    #define nodes on [-1,1] interval(derived above)
    t = np.array([-np.sqrt(3)/3.,np.sqrt(3)/3.])
    #define coefficients (derived in GaussianQuadrature.ipynb)
    c = np.array([1.,1.])
    I = 0
    #now sum up the integral based on the n=2 Gaussian Quadrature formula
    #evaluate f(x(t[j])) with x on the interval [a,b]
    for j in range(n):
        I += (b-a)/2.*c[j]*f(transform_variable(t[j],a,b),*P)
    return I
GQ_n2 = GaussianQ_n2_any_interval(func,0,np.pi,*P)
GQ_n3 = GaussianQ_n3_any_interval(func,0,np.pi,*P)
print("n=2 Quadrature: I = %e, relative error = %e"%(GQ_n2, np.abs(GQ_n2+2)/2))
print("n=3 Quadrature: I = %e, relative error = %e"%(GQ_n3, np.abs(GQ_n3+2)/2))
print("n=4 Quadrature: I = %e, relative error = %e"%(GQ_n4, np.abs(GQ_n4+2)/2))

n=2 Quadrature: I = -2.243950e+00, relative error = 1.219752e-01
n=3 Quadrature: I = -1.991878e+00, relative error = 4.061123e-03
n=4 Quadrature: I = -2.000124e+00, relative error = 6.212054e-05
